In [1]:
import numpy as np
import xarray as xr
import os,errno
import sys
import matplotlib.pyplot as plt
import pandas as pd
import datetime as dt
import salem
import scipy.stats as st
import cartopy.crs as ccrs

dir_data='/Users/ahenny/'
dir2='/Volumes/Extreme Pro/'
dir3='/Volumes/My Passport/'

In [4]:
yrs=np.arange(1958,2024,1)

ds=xr.open_dataset(dir3+'jra55_gridareas.nc')
areas=ds['cell_area']/1e6

ivt_array_polar=[[],[],[],[],[],[],[],[],[],[],[],[]]
ivt_max_array_polar=[[],[],[],[],[],[],[],[],[],[],[],[]]
iwv_array_polar=[[],[],[],[],[],[],[],[],[],[],[],[]]
iwv_max_array_polar=[[],[],[],[],[],[],[],[],[],[],[],[]]
s_array_polar=[[],[],[],[],[],[],[],[],[],[],[],[]]
s_max_array_polar=[[],[],[],[],[],[],[],[],[],[],[],[]]
mfc_array_polar=[[],[],[],[],[],[],[],[],[],[],[],[]]
mfc_max_array_polar=[[],[],[],[],[],[],[],[],[],[],[],[]]

ivt_array_fixed=[[],[],[],[],[],[],[],[],[],[],[],[]]
ivt_max_array_fixed=[[],[],[],[],[],[],[],[],[],[],[],[]]
iwv_array_fixed=[[],[],[],[],[],[],[],[],[],[],[],[]]
iwv_max_array_fixed=[[],[],[],[],[],[],[],[],[],[],[],[]]
s_array_fixed=[[],[],[],[],[],[],[],[],[],[],[],[]]
s_max_array_fixed=[[],[],[],[],[],[],[],[],[],[],[],[]]
mfc_array_fixed=[[],[],[],[],[],[],[],[],[],[],[],[]]
mfc_max_array_fixed=[[],[],[],[],[],[],[],[],[],[],[],[]]

ivt_array_fixed_poleward=[[],[],[],[],[],[],[],[],[],[],[],[]]
ivt_max_array_fixed_poleward=[[],[],[],[],[],[],[],[],[],[],[],[]]
iwv_array_fixed_poleward=[[],[],[],[],[],[],[],[],[],[],[],[]]
iwv_max_array_fixed_poleward=[[],[],[],[],[],[],[],[],[],[],[],[]]
s_array_fixed_poleward=[[],[],[],[],[],[],[],[],[],[],[],[]]
s_max_array_fixed_poleward=[[],[],[],[],[],[],[],[],[],[],[],[]]
mfc_array_fixed_poleward=[[],[],[],[],[],[],[],[],[],[],[],[]]
mfc_max_array_fixed_poleward=[[],[],[],[],[],[],[],[],[],[],[],[]]


ivt_list_polar=[]
ivt_max_list_polar=[]
iwv_list_polar=[]
iwv_max_list_polar=[]
s_list_polar=[]
s_max_list_polar=[]
mfc_list_polar=[]
mfc_max_list_polar=[]

ivt_list_fixed=[]
ivt_max_list_fixed=[]
iwv_list_fixed=[]
iwv_max_list_fixed=[]
s_list_fixed=[]
s_max_list_fixed=[]
mfc_list_fixed=[]
mfc_max_list_fixed=[]

ivt_list_fixed_poleward=[]
ivt_max_list_fixed_poleward=[]
iwv_list_fixed_poleward=[]
iwv_max_list_fixed_poleward=[]
s_list_fixed_poleward=[]
s_max_list_fixed_poleward=[]
mfc_list_fixed_poleward=[]
mfc_max_list_fixed_poleward=[]


for i in range(len(yrs)):
    year=yrs[i]
    print(year)
    
    if 1==1:
        ds=xr.open_dataset(dir2+'jra55.ar.labels.model.variable.'+str(year)+'.nc')
        labeled=ds['ar_labeled']
        
        ds=xr.open_dataset(dir3+'jra55_ivt_'+str(year)+'.nc')

        ivt=ds['ivt'][:,:,:]
        ds=xr.open_dataset(dir3+'jra55_iwv_'+str(year)+'.nc')
        iwv=ds['iwv'][:,:,:]
        ds=xr.open_dataset(dir3+'jra55_u850_'+str(year)+'.nc')
        u=ds['u']
        ds=xr.open_dataset(dir3+'jra55_v850_'+str(year)+'.nc')
        v=ds['v']
        s=np.sqrt(u**2+v**2)
        
        areas['lon']=labeled.lon.values
        ivt['lon']=labeled.lon.values
        iwv['lon']=labeled.lon.values
        areas['lat']=labeled.lat.values
        ivt['lat']=labeled.lat.values
        iwv['lat']=labeled.lat.values
        
        if i==0:
            weights=np.cos(np.deg2rad(labeled.lat))
            weights1=np.cos(np.deg2rad(s.lat))
        
        ivt_list_year=[]
        ivt_max_list_year=[]
        iwv_list_year=[]
        iwv_max_list_year=[]
        s_list_year=[]
        s_max_list_year=[]
        mfc_list_year=[]
        mfc_max_list_year=[]
        
        ivt_array_year=[[],[],[],[],[],[],[],[],[],[],[],[]]
        ivt_max_array_year=[[],[],[],[],[],[],[],[],[],[],[],[]]
        iwv_array_year=[[],[],[],[],[],[],[],[],[],[],[],[]]
        iwv_max_array_year=[[],[],[],[],[],[],[],[],[],[],[],[]]
        s_array_year=[[],[],[],[],[],[],[],[],[],[],[],[]]
        s_max_array_year=[[],[],[],[],[],[],[],[],[],[],[],[]]
        mfc_array_year=[[],[],[],[],[],[],[],[],[],[],[],[]]
        mfc_max_array_year=[[],[],[],[],[],[],[],[],[],[],[],[]]
        
        
        num_list_year=[]
        nums_array_year=[[],[],[],[],[],[],[],[],[],[],[],[]]

        count=0
        for j in range(labeled.time.size):
            #print(j)
            date=pd.to_datetime(labeled.time.values[j])
            month=date.month

            label_sel=labeled[j,:,:]
            unique=np.unique(label_sel.values)
            unique=[x for x in unique if x>0]
            num=len(unique)
            
            ivt_sel=ivt[j,:,:]
            iwv_sel=iwv[j,:,:]
            s_sel=s[j,:,:]
            #mfc_sel=mfc[j,:,:]

            num_day=0
            for k in range(len(unique)):
                sel_feature=label_sel.where(label_sel==unique[k])
                area_feature=areas.where(sel_feature>0)
                area_feature=area_feature.sum(skipna=True).values.tolist()
                
                sel_feature_interp=sel_feature/sel_feature
                sel_feature_interp=sel_feature_interp.fillna(0).interp_like(s_sel)
                sel_feature_interp=sel_feature_interp.where(sel_feature_interp>=0.5)
                sel_feature_interp=sel_feature_interp/sel_feature_interp
                
                if area_feature>0:
                    
                    ivt_sel1=ivt_sel.where(sel_feature>0)
                    ivt_weighted=ivt_sel1.weighted(weights)
                    ivt_mean=ivt_weighted.mean(skipna=True).values.tolist()
                    ivt_max=ivt_sel1.max().values.tolist()
                    
                    iwv_sel1=iwv_sel.where(sel_feature>0)
                    iwv_weighted=iwv_sel1.weighted(weights)
                    iwv_mean=iwv_weighted.mean(skipna=True).values.tolist()
                    iwv_max=iwv_sel1.max().values.tolist()
                    
                    s_sel1=s_sel.where(sel_feature_interp>0)
                    s_weighted=s_sel1.weighted(weights1)
                    s_mean=s_weighted.mean(skipna=True).values.tolist()
                    s_max=s_sel1.max().values.tolist()
                    
                    #mfc_sel1=mfc_sel.where(sel_feature>0)
                    #mfc_weighted=mfc_sel1.weighted(weights)
                    #mfc_mean=mfc_weighted.mean(skipna=True).values.tolist()
                    #mfc_max=mfc_sel1.max().values.tolist()

                    ivt_list_year.append(ivt_mean)
                    ivt_max_list_year.append(ivt_max)
                    iwv_list_year.append(iwv_mean)
                    iwv_max_list_year.append(iwv_max)
                    s_list_year.append(s_mean)
                    s_max_list_year.append(s_max)
                    #mfc_list_year.append(mfc_mean)
                    #mfc_max_list_year.append(mfc_max)
                    
                    if 1==0:
                        fig=plt.figure(figsize=(18,12))
                        ax=plt.subplot(1,1,1,projection=ccrs.PlateCarree())
                        ax.coastlines(resolution='10m')
                        ax.pcolormesh(s_sel1.lon.values,s_sel1.lat.values,s_sel1,vmin=0,vmax=20,cmap=plt.cm.viridis)
                        plt.show()

                        #fig=plt.figure(figsize=(18,12))
                        #ax=plt.subplot(1,1,1,projection=ccrs.PlateCarree())
                        #ax.coastlines(resolution='10m')
                        #ax.pcolormesh(mfc_sel.lon.values,mfc_sel.lat.values,mfc_sel1,vmin=-20e-5,vmax=20e-5,cmap=plt.cm.seismic)
                        #plt.show()

                    ivt_array_year[month-1].append(ivt_mean)
                    ivt_max_array_year[month-1].append(ivt_max)
                    iwv_array_year[month-1].append(iwv_mean)
                    iwv_max_array_year[month-1].append(iwv_max)
                    s_array_year[month-1].append(s_mean)
                    s_max_array_year[month-1].append(s_max)
                    #mfc_array_year[month-1].append(mfc_mean)
                    #mfc_max_array_year[month-1].append(mfc_max)
                    
                    num_day=num_day+1
                    
                    if 1==0:
                        fig=plt.figure(figsize=(18,12))
                        ax=plt.subplot(1,1,1,projection=ccrs.PlateCarree())
                        ax.coastlines(resolution='10m')
                        ax.pcolormesh(sel_feature.lon.values,sel_feature.lat.values,ivt_sel1,
                                     vmin=0,vmax=700,cmap=plt.cm.Greys)

                        print('IVT:'+str(ivt_mean)+','+str(ivt_max))
                        print('IWV:'+str(iwv_mean)+','+str(iwv_max))
                        print('S:'+str(s_mean)+','+str(s_max))
                        plt.show()
                        
                        fig=plt.figure(figsize=(18,12))
                        ax=plt.subplot(1,1,1,projection=ccrs.PlateCarree())
                        ax.coastlines(resolution='10m')
                        ax.pcolormesh(sel_feature.lon.values,sel_feature.lat.values,iwv_sel1,
                                     vmin=0,vmax=60,cmap=plt.cm.Greens)

                        plt.show()

            num_list_year.append(num_day)
            nums_array_year[month-1].append(num_day)
        #if j==10:
        #    sys.exit()
        s_list_year=[x for x in s_list_year if x>=0]
        s_max_list_year=[x for x in s_max_list_year if x>=0]
        
        mean_ivt=float(sum(ivt_list_year))/float(len(ivt_list_year))
        max_ivt=float(sum(ivt_max_list_year))/float(len(ivt_max_list_year))
        mean_iwv=float(sum(iwv_list_year))/float(len(iwv_list_year))
        max_iwv=float(sum(iwv_max_list_year))/float(len(iwv_max_list_year))
        mean_s=float(sum(s_list_year))/float(len(s_list_year))
        max_s=float(sum(s_max_list_year))/float(len(s_max_list_year))
        #mean_mfc=float(sum(mfc_list_year))/float(len(mfc_list_year))
        #max_mfc=float(sum(mfc_max_list_year))/float(len(mfc_max_list_year))
        
        ivt_list_polar.append(mean_ivt)
        ivt_max_list_polar.append(max_ivt)
        iwv_list_polar.append(mean_iwv)
        iwv_max_list_polar.append(max_iwv)
        s_list_polar.append(mean_s)
        s_max_list_polar.append(max_s)
        #mfc_list_polar.append(mean_mfc)
        #mfc_max_list_polar.append(max_mfc)
        
        
        for j in range(len(nums_array_year)):
            ivt_array_polar[j].append(float(sum(ivt_array_year[j]))/float(len(ivt_array_year[j])))
            ivt_max_array_polar[j].append(float(sum(ivt_max_array_year[j]))/float(len(ivt_max_array_year[j])))
        
            iwv_array_polar[j].append(float(sum(iwv_array_year[j]))/float(len(iwv_array_year[j])))
            iwv_max_array_polar[j].append(float(sum(iwv_max_array_year[j]))/float(len(iwv_max_array_year[j])))
        
            s_array_polar[j].append(float(sum(s_array_year[j]))/float(len(s_array_year[j])))
            s_max_array_polar[j].append(float(sum(s_max_array_year[j]))/float(len(s_max_array_year[j])))

            #mfc_array_polar[j].append(float(sum(mfc_array_year[j]))/float(len(mfc_array_year[j])))
            #mfc_max_array_polar[j].append(float(sum(mfc_max_array_year[j]))/float(len(mfc_max_array_year[j])))

        print('A')
        #################

        ds=xr.open_dataset(dir_data+'jra55.ar.labels.model.'+str(year)+'.nc')
        labeled=ds['ar_labeled'][:,:,:]

        ivt_list_year=[]
        ivt_max_list_year=[]
        iwv_list_year=[]
        iwv_max_list_year=[]
        s_list_year=[]
        s_max_list_year=[]
        mfc_list_year=[]
        mfc_max_list_year=[]
        
        ivt_array_year=[[],[],[],[],[],[],[],[],[],[],[],[]]
        ivt_max_array_year=[[],[],[],[],[],[],[],[],[],[],[],[]]
        iwv_array_year=[[],[],[],[],[],[],[],[],[],[],[],[]]
        iwv_max_array_year=[[],[],[],[],[],[],[],[],[],[],[],[]]
        s_array_year=[[],[],[],[],[],[],[],[],[],[],[],[]]
        s_max_array_year=[[],[],[],[],[],[],[],[],[],[],[],[]]
        mfc_array_year=[[],[],[],[],[],[],[],[],[],[],[],[]]
        mfc_max_array_year=[[],[],[],[],[],[],[],[],[],[],[],[]]
        
        num_list_year=[]
        nums_array_year=[[],[],[],[],[],[],[],[],[],[],[],[]]

        count=0
        for j in range(labeled.time.size):
            #print(j)
            date=pd.to_datetime(labeled.time.values[j])
            month=date.month

            label_sel=labeled[j,:,:]
            unique=np.unique(label_sel.values)
            unique=[x for x in unique if x>0]
            num=len(unique)
            
            ivt_sel=ivt[j,:,:]
            iwv_sel=iwv[j,:,:]
            s_sel=s[j,:,:]
            #mfc_sel=mfc[j,:,:]

            num_day=0
            for k in range(len(unique)):
                sel_feature=label_sel.where(label_sel==unique[k])
                area_feature=areas.where(sel_feature>0)
                area_feature=area_feature.sum(skipna=True).values.tolist()
                
                sel_feature_interp=sel_feature/sel_feature
                sel_feature_interp=sel_feature_interp.fillna(0).interp_like(s_sel)
                sel_feature_interp=sel_feature_interp.where(sel_feature_interp>=0.5)
                sel_feature_interp=sel_feature_interp/sel_feature_interp

                if area_feature>0:
                    ivt_sel1=ivt_sel.where(sel_feature>0)
                    ivt_weighted=ivt_sel1.weighted(weights)
                    ivt_mean=ivt_weighted.mean(skipna=True).values.tolist()
                    ivt_max=ivt_sel1.max().values.tolist()
                    
                    iwv_sel1=iwv_sel.where(sel_feature>0)
                    iwv_weighted=iwv_sel1.weighted(weights)
                    iwv_mean=iwv_weighted.mean(skipna=True).values.tolist()
                    iwv_max=iwv_sel1.max().values.tolist()
                    
                    s_sel1=s_sel.where(sel_feature_interp>0)
                    s_weighted=s_sel1.weighted(weights1)
                    s_mean=s_weighted.mean(skipna=True).values.tolist()
                    s_max=s_sel1.max().values.tolist()
                    
                    #mfc_sel1=mfc_sel.where(sel_feature>0)
                    #mfc_weighted=mfc_sel1.weighted(weights)
                    #mfc_mean=mfc_weighted.mean(skipna=True).values.tolist()
                    #mfc_max=mfc_sel1.max().values.tolist()

                    ivt_list_year.append(ivt_mean)
                    ivt_max_list_year.append(ivt_max)
                    iwv_list_year.append(iwv_mean)
                    iwv_max_list_year.append(iwv_max)
                    s_list_year.append(s_mean)
                    s_max_list_year.append(s_max)
                    #mfc_list_year.append(mfc_mean)
                    #mfc_max_list_year.append(mfc_max)
                    
                    
                    ivt_array_year[month-1].append(ivt_mean)
                    ivt_max_array_year[month-1].append(ivt_max)
                    iwv_array_year[month-1].append(iwv_mean)
                    iwv_max_array_year[month-1].append(iwv_max)
                    s_array_year[month-1].append(s_mean)
                    s_max_array_year[month-1].append(s_max)
                    #mfc_array_year[month-1].append(mfc_mean)
                    #mfc_max_array_year[month-1].append(mfc_max)
                        
                    num_day=num_day+1

            num_list_year.append(num_day)
            nums_array_year[month-1].append(num_day)

        s_list_year=[x for x in s_list_year if x>=0]
        s_max_list_year=[x for x in s_max_list_year if x>=0]
        
        mean_ivt=float(sum(ivt_list_year))/float(len(ivt_list_year))
        max_ivt=float(sum(ivt_max_list_year))/float(len(ivt_max_list_year))
        mean_iwv=float(sum(iwv_list_year))/float(len(iwv_list_year))
        max_iwv=float(sum(iwv_max_list_year))/float(len(iwv_max_list_year))
        mean_s=float(sum(s_list_year))/float(len(s_list_year))
        max_s=float(sum(s_max_list_year))/float(len(s_max_list_year))
        #mean_mfc=float(sum(mfc_list_year))/float(len(mfc_list_year))
        #max_mfc=float(sum(mfc_max_list_year))/float(len(mfc_max_list_year))
        
        ivt_list_fixed.append(mean_ivt)
        ivt_max_list_fixed.append(max_ivt)
        iwv_list_fixed.append(mean_iwv)
        iwv_max_list_fixed.append(max_iwv)
        s_list_fixed.append(mean_s)
        s_max_list_fixed.append(max_s)
        #mfc_list_fixed.append(mean_mfc)
        #mfc_max_list_fixed.append(max_mfc)
        
        for j in range(len(nums_array_year)):
            ivt_array_fixed[j].append(float(sum(ivt_array_year[j]))/float(len(ivt_array_year[j])))
            ivt_max_array_fixed[j].append(float(sum(ivt_max_array_year[j]))/float(len(ivt_max_array_year[j])))
        
            iwv_array_fixed[j].append(float(sum(iwv_array_year[j]))/float(len(iwv_array_year[j])))
            iwv_max_array_fixed[j].append(float(sum(iwv_max_array_year[j]))/float(len(iwv_max_array_year[j])))
        
            s_array_fixed[j].append(float(sum(s_array_year[j]))/float(len(s_array_year[j])))
            s_max_array_fixed[j].append(float(sum(s_max_array_year[j]))/float(len(s_max_array_year[j])))

            #mfc_array_fixed[j].append(float(sum(mfc_array_year[j]))/float(len(mfc_array_year[j])))
            #mfc_max_array_fixed[j].append(float(sum(mfc_max_array_year[j]))/float(len(mfc_max_array_year[j])))

        print('B')
        
        ds=xr.open_dataset(dir2+'jra55.ar.labels.model.poleward.'+str(year)+'.nc')
        labeled=ds['ar_labeled']

        ivt_list_year=[]
        ivt_max_list_year=[]
        iwv_list_year=[]
        iwv_max_list_year=[]
        s_list_year=[]
        s_max_list_year=[]
        mfc_list_year=[]
        mfc_max_list_year=[]
        
        ivt_array_year=[[],[],[],[],[],[],[],[],[],[],[],[]]
        ivt_max_array_year=[[],[],[],[],[],[],[],[],[],[],[],[]]
        iwv_array_year=[[],[],[],[],[],[],[],[],[],[],[],[]]
        iwv_max_array_year=[[],[],[],[],[],[],[],[],[],[],[],[]]
        s_array_year=[[],[],[],[],[],[],[],[],[],[],[],[]]
        s_max_array_year=[[],[],[],[],[],[],[],[],[],[],[],[]]
        mfc_array_year=[[],[],[],[],[],[],[],[],[],[],[],[]]
        mfc_max_array_year=[[],[],[],[],[],[],[],[],[],[],[],[]]
        
        num_list_year=[]
        nums_array_year=[[],[],[],[],[],[],[],[],[],[],[],[]]

        count=0
        for j in range(labeled.time.size):
            #print(j)
            date=pd.to_datetime(labeled.time.values[j])
            month=date.month

            label_sel=labeled[j,:,:]
            unique=np.unique(label_sel.values)
            unique=[x for x in unique if x>0]
            num=len(unique)
            
            ivt_sel=ivt[j,:,:]
            iwv_sel=iwv[j,:,:]
            s_sel=s[j,:,:]
            #mfc_sel=mfc[j,:,:]

            num_day=0
            for k in range(len(unique)):
                sel_feature=label_sel.where(label_sel==unique[k])
                area_feature=areas.where(sel_feature>0)
                area_feature=area_feature.sum(skipna=True).values.tolist()
                
                sel_feature_interp=sel_feature/sel_feature
                sel_feature_interp=sel_feature_interp.fillna(0).interp_like(s_sel)
                sel_feature_interp=sel_feature_interp.where(sel_feature_interp>=0.5)
                sel_feature_interp=sel_feature_interp/sel_feature_interp

                if area_feature>0:
                    
                    ivt_sel1=ivt_sel.where(sel_feature>0)
                    ivt_weighted=ivt_sel1.weighted(weights)
                    ivt_mean=ivt_weighted.mean(skipna=True).values.tolist()
                    ivt_max=ivt_sel1.max().values.tolist()
                    
                    iwv_sel1=iwv_sel.where(sel_feature>0)
                    iwv_weighted=iwv_sel1.weighted(weights)
                    iwv_mean=iwv_weighted.mean(skipna=True).values.tolist()
                    iwv_max=iwv_sel1.max().values.tolist()
                    
                    s_sel1=s_sel.where(sel_feature_interp>0)
                    s_weighted=s_sel1.weighted(weights1)
                    s_mean=s_weighted.mean(skipna=True).values.tolist()
                    s_max=s_sel1.max().values.tolist()
                    
                    #mfc_sel1=mfc_sel.where(sel_feature>0)
                    #mfc_weighted=mfc_sel1.weighted(weights)
                    #mfc_mean=mfc_weighted.mean(skipna=True).values.tolist()
                    #mfc_max=mfc_sel1.max().values.tolist()

                    ivt_list_year.append(ivt_mean)
                    ivt_max_list_year.append(ivt_max)
                    iwv_list_year.append(iwv_mean)
                    iwv_max_list_year.append(iwv_max)
                    s_list_year.append(s_mean)
                    s_max_list_year.append(s_max)
                    #mfc_list_year.append(mfc_mean)
                    #mfc_max_list_year.append(mfc_max)
                    
                    ivt_array_year[month-1].append(ivt_mean)
                    ivt_max_array_year[month-1].append(ivt_max)
                    iwv_array_year[month-1].append(iwv_mean)
                    iwv_max_array_year[month-1].append(iwv_max)
                    s_array_year[month-1].append(s_mean)
                    s_max_array_year[month-1].append(s_max)
                    #mfc_array_year[month-1].append(mfc_mean)
                    #mfc_max_array_year[month-1].append(mfc_max)
                        
                    num_day=num_day+1

            num_list_year.append(num_day)
            nums_array_year[month-1].append(num_day)

        s_list_year=[x for x in s_list_year if x>=0]
        s_max_list_year=[x for x in s_max_list_year if x>=0]
        
        mean_ivt=float(sum(ivt_list_year))/float(len(ivt_list_year))
        max_ivt=float(sum(ivt_max_list_year))/float(len(ivt_max_list_year))
        mean_iwv=float(sum(iwv_list_year))/float(len(iwv_list_year))
        max_iwv=float(sum(iwv_max_list_year))/float(len(iwv_max_list_year))
        mean_s=float(sum(s_list_year))/float(len(s_list_year))
        max_s=float(sum(s_max_list_year))/float(len(s_max_list_year))
        #mean_mfc=float(sum(mfc_list_year))/float(len(mfc_list_year))
        #max_mfc=float(sum(mfc_max_list_year))/float(len(mfc_max_list_year))
        
        ivt_list_fixed_poleward.append(mean_ivt)
        ivt_max_list_fixed_poleward.append(max_ivt)
        iwv_list_fixed_poleward.append(mean_iwv)
        iwv_max_list_fixed_poleward.append(max_iwv)
        s_list_fixed_poleward.append(mean_s)
        s_max_list_fixed_poleward.append(max_s)
        #mfc_list_fixed_poleward.append(mean_mfc)
        #mfc_max_list_fixed_poleward.append(max_mfc)
        
        for j in range(len(nums_array_year)):
            ivt_array_fixed_poleward[j].append(float(sum(ivt_array_year[j]))/float(len(ivt_array_year[j])))
            ivt_max_array_fixed_poleward[j].append(float(sum(ivt_max_array_year[j]))/float(len(ivt_max_array_year[j])))
        
            iwv_array_fixed_poleward[j].append(float(sum(iwv_array_year[j]))/float(len(iwv_array_year[j])))
            iwv_max_array_fixed_poleward[j].append(float(sum(iwv_max_array_year[j]))/float(len(iwv_max_array_year[j])))
        
            s_array_fixed_poleward[j].append(float(sum(s_array_year[j]))/float(len(s_array_year[j])))
            s_max_array_fixed_poleward[j].append(float(sum(s_max_array_year[j]))/float(len(s_max_array_year[j])))

            #mfc_array_fixed_poleward[j].append(float(sum(mfc_array_year[j]))/float(len(mfc_array_year[j])))
            #mfc_max_array_fixed_poleward[j].append(float(sum(mfc_max_array_year[j]))/float(len(mfc_max_array_year[j])))          
       
    print('C')

1958
A
B
C
1959
A
B
C
1960
A
B
C
1961
A
B
C
1962
A
B
C
1963
A
B
C
1964
A
B
C
1965
A
B
C
1966
A
B
C
1967
A
B
C
1968
A
B
C
1969
A
B
C
1970
A
B
C
1971
A
B
C
1972
A
B
C
1973
A
B
C
1974
A
B
C
1975
A
B
C
1976
A
B
C
1977
A
B
C
1978
A
B
C
1979
A
B
C
1980
A
B
C
1981
A
B
C
1982
A
B
C
1983
A
B
C
1984
A
B
C
1985
A
B
C
1986
A
B
C
1987
A
B
C
1988
A
B
C
1989
A
B
C
1990
A
B
C
1991
A
B
C
1992
A
B
C
1993
A
B
C
1994
A
B
C
1995
A
B
C
1996
A
B
C
1997
A
B
C
1998
A
B
C
1999
A
B
C
2000
A
B
C
2001
A
B
C
2002
A
B
C
2003
A
B
C
2004
A
B
C
2005
A
B
C
2006
A
B
C
2007
A
B
C
2008
A
B
C
2009
A
B
C
2010
A
B
C
2011
A
B
C
2012
A
B
C
2013
A
B
C
2014
A
B
C
2015
A
B
C
2016
A
B
C
2017
A
B
C
2018
A
B
C
2019
A
B
C
2020
A
B
C
2021
A
B
C
2022
A
B
C
2023
A
B
C


In [6]:
dk=xr.Dataset()

dk['ivt_array_jra55_polar']=(('months','years'),np.asarray(ivt_array_polar))
dk['ivt_max_array_jra55_polar']=(('months','years'),np.asarray(ivt_max_array_polar))
dk['iwv_array_jra55_polar']=(('months','years'),np.asarray(iwv_array_polar))
dk['iwv_max_array_jra55_polar']=(('months','years'),np.asarray(iwv_max_array_polar))
dk['s_array_jra55_polar']=(('months','years'),np.asarray(s_array_polar))
dk['s_max_array_jra55_polar']=(('months','years'),np.asarray(s_max_array_polar))
#dk['mfc_array_jra55_polar']=(('months','years'),np.asarray(mfc_array_polar))
#dk['mfc_max_array_jra55_polar']=(('months','years'),np.asarray(mfc_max_array_polar))

dk['ivt_array_jra55_fixed']=(('months','years'),np.asarray(ivt_array_fixed))
dk['ivt_max_array_jra55_fixed']=(('months','years'),np.asarray(ivt_max_array_fixed))
dk['iwv_array_jra55_fixed']=(('months','years'),np.asarray(iwv_array_fixed))
dk['iwv_max_array_jra55_fixed']=(('months','years'),np.asarray(iwv_max_array_fixed))
dk['s_array_jra55_fixed']=(('months','years'),np.asarray(s_array_fixed))
dk['s_max_array_jra55_fixed']=(('months','years'),np.asarray(s_max_array_fixed))
#dk['mfc_array_jra55_fixed']=(('months','years'),np.asarray(mfc_array_fixed))
#dk['mfc_max_array_jra55_fixed']=(('months','years'),np.asarray(mfc_max_array_fixed))

dk['ivt_array_jra55_fixed_poleward']=(('months','years'),np.asarray(ivt_array_fixed_poleward))
dk['ivt_max_array_jra55_fixed_poleward']=(('months','years'),np.asarray(ivt_max_array_fixed_poleward))
dk['iwv_array_jra55_fixed_poleward']=(('months','years'),np.asarray(iwv_array_fixed_poleward))
dk['iwv_max_array_jra55_fixed_poleward']=(('months','years'),np.asarray(iwv_max_array_fixed_poleward))
dk['s_array_jra55_fixed_poleward']=(('months','years'),np.asarray(s_array_fixed_poleward))
dk['s_max_array_jra55_fixed_poleward']=(('months','years'),np.asarray(s_max_array_fixed_poleward))
#dk['mfc_array_jra55_fixed_poleward']=(('months','years'),np.asarray(mfc_array_fixed_poleward))
#dk['mfc_max_array_jra55_fixed_poleward']=(('months','years'),np.asarray(mfc_max_array_fixed_poleward))

dk['ivt_list_jra55_polar']=(('years'),ivt_list_polar)
dk['ivt_max_list_jra55_polar']=(('years'),ivt_max_list_polar)
dk['iwv_list_jra55_polar']=(('years'),iwv_list_polar)
dk['iwv_max_list_jra55_polar']=(('years'),iwv_max_list_polar)
dk['s_list_jra55_polar']=(('years'),s_list_polar)
dk['s_max_list_jra55_polar']=(('years'),s_max_list_polar)
#dk['mfc_list_jra55_polar']=(('years'),mfc_list_polar)
#dk['mfc_max_list_jra55_polar']=(('years'),mfc_max_list_polar)

dk['ivt_list_jra55_fixed']=(('years'),ivt_list_fixed)
dk['ivt_max_list_jra55_fixed']=(('years'),ivt_max_list_fixed)
dk['iwv_list_jra55_fixed']=(('years'),iwv_list_fixed)
dk['iwv_max_list_jra55_fixed']=(('years'),iwv_max_list_fixed)
dk['s_list_jra55_fixed']=(('years'),s_list_fixed)
dk['s_max_list_jra55_fixed']=(('years'),s_max_list_fixed)
#dk['mfc_list_jra55_fixed']=(('years'),mfc_list_fixed)
#dk['mfc_max_list_jra55_fixed']=(('years'),mfc_max_list_fixed)

dk['ivt_list_jra55_fixed_poleward']=(('years'),ivt_list_fixed_poleward)
dk['ivt_max_list_jra55_fixed_poleward']=(('years'),ivt_max_list_fixed_poleward)
dk['iwv_list_jra55_fixed_poleward']=(('years'),iwv_list_fixed_poleward)
dk['iwv_max_list_jra55_fixed_poleward']=(('years'),iwv_max_list_fixed_poleward)
dk['s_list_jra55_fixed_poleward']=(('years'),s_list_fixed_poleward)
dk['s_max_list_jra55_fixed_poleward']=(('years'),s_max_list_fixed_poleward)
#dk['mfc_list_jra55_fixed_poleward']=(('years'),mfc_list_fixed_poleward)
#dk['mfc_max_list_jra55_fixed_poleward']=(('years'),mfc_max_list_fixed_poleward)

dk.coords['years']=np.arange(1958,2024,1)
dk.coords['months']=np.arange(1,13,1)
try:
    os.remove(dir_data+'ar_chars_synoptic_jra55.nc')
except OSError:
    pass
dk.to_netcdf(dir_data+'ar_chars_synoptic_jra55.nc',mode='w',format='NETCDF4')